In [1]:
#install
!pip install keras
!pip install transformers

In [2]:
# import
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [3]:
# year 마다 데이터 불러와서 변수 생성

# year_list = ['15','16','17','18','19','20']
# for year in year_list:
#   globals()[f'data_{year}'] = pd.read_csv(f'/content/drive/MyDrive/politics_data/{year}대 민주당_보수당 구분.csv')

data = pd.read_csv('/content/drive/MyDrive/politics_data/15대 민주당_보수당 구분.csv')

15대 먼저 돌려보자

In [4]:

data.head(10)

,L|R,Content
0,1,의석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제5차 2002년월드컵등국제경기대...
1,1,의사일정에 들어가기 전에 위원장의 신상에 관한 발언을 하고자 합니다. 존경하는 위원...
2,1,신한국당에서 간사직을 맡고 있는 김기수 위원입니다. 조금 전에 위원장께서 말씀을 드...
3,1,그러면 의사일정 제1항 위원장(신경식) 사임의 건을 상정하겠습니다. 신경식 위원장님...
4,1,다음은 의사일정 제2항 위원장 선임의 건을 상정하겠습니다. 위원 여러분께서 잘 아시...
5,0,새정치국민회의 소속 박광태 위원입니다. 위원장 선임방법은 사회자께서도 말씀하셨다시피...
6,1,지금 존경하는 박광태 위원께서 위원장 선임방법에 대하여 구두호천에 의해서 선출하자는...
7,0,새정치국민회의 최재승 위원입니다. 우리 위원회는 본래 설치한 목적이 한국적인 국제경...
8,1,방금 최재승 위원으로부터 김진재 위원을 위원장으로 선임하자는 추천이 있었습니다. 다...
9,1,먼저 여러 가지로 부족한 점이 많은 이 사람을 만장일치로 위원장으로 선출해 주신 선...


In [5]:
data.columns

Index(['L|R', 'Content'], dtype='object')

In [6]:
# gpu (cuda)
if torch.cuda.is_available():
  DEVICE = torch.device("cuda")
else:
  DEVICE= torch.device("cpu")

print(DEVICE)


cuda


EDA & 전처리


In [7]:
data.isnull().sum()


L|R        0
Content    1
dtype: int64

In [8]:
data[data['Content'].isnull()]

,L|R,Content
221670,1,NaN


In [9]:
# 삭제
data.dropna(axis=0,how='any',inplace=True)

data.isnull().sum()

L|R        0
Content    0
dtype: int64

In [10]:
# 칼럼명 & 순서 변경
data.columns = ['label','content']
data = data[['content','label']]



In [11]:
# label 비율 확인
data['label'].value_counts(normalize=True)

1    0.618191
0    0.381809
Name: label, dtype: float64

In [12]:
data.head(10)

,content,label
0,의석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제5차 2002년월드컵등국제경기대...,1
1,의사일정에 들어가기 전에 위원장의 신상에 관한 발언을 하고자 합니다. 존경하는 위원...,1
2,신한국당에서 간사직을 맡고 있는 김기수 위원입니다. 조금 전에 위원장께서 말씀을 드...,1
3,그러면 의사일정 제1항 위원장(신경식) 사임의 건을 상정하겠습니다. 신경식 위원장님...,1
4,다음은 의사일정 제2항 위원장 선임의 건을 상정하겠습니다. 위원 여러분께서 잘 아시...,1
5,새정치국민회의 소속 박광태 위원입니다. 위원장 선임방법은 사회자께서도 말씀하셨다시피...,0
6,지금 존경하는 박광태 위원께서 위원장 선임방법에 대하여 구두호천에 의해서 선출하자는...,1
7,새정치국민회의 최재승 위원입니다. 우리 위원회는 본래 설치한 목적이 한국적인 국제경...,0
8,방금 최재승 위원으로부터 김진재 위원을 위원장으로 선임하자는 추천이 있었습니다. 다...,1
9,먼저 여러 가지로 부족한 점이 많은 이 사람을 만장일치로 위원장으로 선출해 주신 선...,1


In [13]:
train, test = train_test_split(data,test_size=0.2)
print(train.shape)
print(test.shape)

(321675, 2)
(80419, 2)


전처리

In [14]:
# BERT 입력 형식에 맞게 변환
document_bert = ["[CLS] " + str(s) + " [SEP]" for s in train.content]
document_bert[:5]

['[CLS] 그것이 몇 조냐 말이에요. 아는 사람 없어요? [SEP]',
 '[CLS] 지금 이해구 위원께서 외형적으로 좋게 말씀하셨는데 사실상 준설때문에 일어나는 농조 안의 비리가 엄청나게 있는 것으로 파악하고 있습니다. 지금 농조에 대한 감독권이 1차적으로 시도에 있고 농림수산부에서 직접 하는 것입니까? 문제점이 있으면 누가 감사하고 누가 지적하고 그럽니까? [SEP]',
 '[CLS] 알겠습니다. [SEP]',
 '[CLS] 의사일정에 들어가기 전에 지금 입법조사관이 보고하신 대로 존경하는 이우재 위원께서 우리 위원회에 보임되어 오셨습니다. 인사 말씀해 주시기 바랍니다. [SEP]',
 '[CLS] 지금 얘기가 직접 나왔으니까 지금 그런 것 기아의 퇴출 부도처리 또 제일은행 같은 것 이것을 현실적으로 그렇게 할 수 없었다 하는 것은 증인의 판단입니까? 아니면…… [SEP]']

In [15]:
# 토크나이징
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in document_bert]


In [77]:
print(document_bert[0])
print(tokenized_texts[0])

[CLS] 채 위원! 종합적으로 왜 손을 대지 못하는 상황에 있었는가 그것을 설명하니까 그 설명을 다 들으시고 질의해 주세요. 계속 답변해 주세요. [SEP]
['[CLS]', '채', '위', '##원', '!', '종', '##합', '##적으로', '왜', '손', '##을', '대', '##지', '못', '##하는', '상', '##황', '##에', '있', '##었', '##는', '##가', '그', '##것을', '설', '##명', '##하', '##니', '##까', '그', '설', '##명을', '다', '들', '##으', '##시', '##고', '질', '##의', '##해', '주', '##세', '##요', '.', '계속', '답', '##변', '##해', '주', '##세', '##요', '.', '[SEP]']


In [20]:
# 라벨 추출
labels = train['label'].values
labels


array([1, 1, 1, ..., 1, 0, 0])

In [17]:
#패딩

# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   8924,  97403,   9282,   9678, 118728,   9251,  10739,
        10530,  48549,    119,   9519,  11018,   9405,  61250,   9555,
        12965,  48549,    136,    102,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [18]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [21]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels,
                                                                                    random_state=2018,
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks,
                                                       input_ids,
                                                       random_state=2018,
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,   9663,  11018,   8924,  93834,   9251,  12178,   8869,  48345,
           119,   9485,  25549,  21611, 103155,  44359,  19105,   8887, 103155,
         10622,  12308,  10954,  16367,  42784,  10530,   9323,  37824,  11513,
          9965,  12965,  48549,    119,   8924,  37388, 118748,  11664,  25805,
         17206, 105946,   9764,  22200,  17022,  86015,  18392,  93835,  12749,
         18392,  11261,   9612,  14871,  85386,   8852,  11664,   9186,  48653,
          9460,  19105,  12428,  21789,  11489,  10348,  21789,   9186,  48653,
         11261,   9692, 119147,  41521,  90537,  48549,    119,   8924,  30873,
         25503, 118671,   8924,  82838,   9251, 119106,  10622,   9460,  16605,
         70146,   9960,  87281,  25503, 118671,   9685, 119081,  48345,    119,
          9546, 118837,  90537,   9246,  16323,  25258,   9574,  79544,  10739,
         24335,  60362,  27487,   9706,  40032,   9998,  13890, 108578,   9251,
        119106, 102246,  12092,   8924, 

In [22]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

전처리 test (train 과 동일)


In [25]:
# 리뷰 문장 추출
sentences = test['content']
sentences[:10]

303436    금강산 관광객 민영미 씨 억류사건은 우리에게 상당한 충격을 주고 있습니다. 햇볕정책...
252073    그러면 지금 솔담배가 시골에 가면 나오는 날은 대판 싸움이 납니다. 서로 그 담배 ...
326585                                그것이 언제라고 말씀하실 수 있습니까?
269804    발언을 하고 있어요. 남의 얘기를 듣지도 않고 그럽니까? 공정거래위원장이 판단해서 ...
192844               수고하셨습니다. 다음은 존경하는 신낙균 위원 질의해 주시기 바랍니다.
176837    그러면 이렇게 하시지요. 지금 아마 답변 내용이 조금 남아 있는지 모르겠는데 전부 ...
137451                                      그것을 어떻게 처리했습니까?
285569    이 문제에 대해서 이따가 박광태 위원 말씀대로 답변하시는 도중에 박광태 위원의 질의...
73502                             제가 자료제출 요구를 한다고 말하지 않았어요?
299110    그 한 시간이라는 것은 지금 누구한테 물어보겠다는 것입니까? 형무소장한테 물어보는 ...
Name: content, dtype: object

In [26]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 금강산 관광객 민영미 씨 억류사건은 우리에게 상당한 충격을 주고 있습니다. 햇볕정책 1년 4개월만에 또 하나의 시련에 부딪혔다고 봅니다. 우리가 또 이 문제해결을 위해서 최우선적으로 총력을 경주해야 한다고 봅니다. 그러나 우리가 곰곰히 따지고 보면 햇볕정책 때문에 이런 억류사건이 필연적으로 일어난 것은 아니라고 봅니다. 왜냐 하면 우리가 최근 수년 동안 북한이 이러한 억류를 한 사례를 몇 가지 본다면 구정권때인 95년 8월에도 쌀 지원을 위해서 청진항에 입항한 삼선비너스호 1등 항해사 이양천 씨가 청진항 주변 사진을 찍었다는 단순한 이유로 8일 동안이나 억류를 하고 1000달러의 벌금을 물려서 석방을 했습니다. 또 97년 10월 이것도 역시 김영삼 정권때 일입니다마는 김정일 위원장의 사진이 실린 로동신문을 찢어서 휴지통에 넣었다는 이유로 경수로 공사현장의 근로자들을 그 당시 집단적으로 숙소에 5일간이나 감금을 했다가 우리가 재발방지를 다짐하고 항의를 하니까 석방시킨 사례도 있었습니다. 또 세번째로는 94년 8월에 북한의 중국 국경지역인 훈춘지역 교포 상대로 의류장사를 하던 중국동포 배용문 이상찬 씨를 북한측에서 20일간이나 함북 온성에 억류시켰다가 석방시킨 사건도 있었습니다. 또 최근에는 미국 시민권자인 김진경 연변 과기대 총장을 기독교 전파 등의 이유로 43일간이나 억류시켰던 일이 있었습니다. 그래서 햇볕정책 1년 4개월의 필연적인 부산물로서 이런 억류사건이 일어났다고는 보지 않습니다. 그러나 어떻든 정부가 대북 포용정책을 취하는 과정에서 더구나 금강산 관광중에서 일어난 일이기 때문에 우리 정부에서는 최우선적으로 이 문제 해결을 위해서 총력을 경주해야 하는데 오늘 보고된 내용은 너무 현대측에 실질적으로 모든 것을 일임하고 있지 않느냐 하는 인상을 지울 수 없는데 좀더 총체적이고 강력한 석방대책을 세워서 추진해야 된다고 보는데 장관께서는 어떻게 생각하십니까? 간략하게 답변 주시지요. [SEP]',
 '[CLS] 그러면 지금 솔담배가 시골에 가면 

In [27]:
# 라벨 추출
labels = test['label'].values
labels

array([1, 1, 0, ..., 0, 0, 1])

In [28]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 금강산 관광객 민영미 씨 억류사건은 우리에게 상당한 충격을 주고 있습니다. 햇볕정책 1년 4개월만에 또 하나의 시련에 부딪혔다고 봅니다. 우리가 또 이 문제해결을 위해서 최우선적으로 총력을 경주해야 한다고 봅니다. 그러나 우리가 곰곰히 따지고 보면 햇볕정책 때문에 이런 억류사건이 필연적으로 일어난 것은 아니라고 봅니다. 왜냐 하면 우리가 최근 수년 동안 북한이 이러한 억류를 한 사례를 몇 가지 본다면 구정권때인 95년 8월에도 쌀 지원을 위해서 청진항에 입항한 삼선비너스호 1등 항해사 이양천 씨가 청진항 주변 사진을 찍었다는 단순한 이유로 8일 동안이나 억류를 하고 1000달러의 벌금을 물려서 석방을 했습니다. 또 97년 10월 이것도 역시 김영삼 정권때 일입니다마는 김정일 위원장의 사진이 실린 로동신문을 찢어서 휴지통에 넣었다는 이유로 경수로 공사현장의 근로자들을 그 당시 집단적으로 숙소에 5일간이나 감금을 했다가 우리가 재발방지를 다짐하고 항의를 하니까 석방시킨 사례도 있었습니다. 또 세번째로는 94년 8월에 북한의 중국 국경지역인 훈춘지역 교포 상대로 의류장사를 하던 중국동포 배용문 이상찬 씨를 북한측에서 20일간이나 함북 온성에 억류시켰다가 석방시킨 사건도 있었습니다. 또 최근에는 미국 시민권자인 김진경 연변 과기대 총장을 기독교 전파 등의 이유로 43일간이나 억류시켰던 일이 있었습니다. 그래서 햇볕정책 1년 4개월의 필연적인 부산물로서 이런 억류사건이 일어났다고는 보지 않습니다. 그러나 어떻든 정부가 대북 포용정책을 취하는 과정에서 더구나 금강산 관광중에서 일어난 일이기 때문에 우리 정부에서는 최우선적으로 이 문제 해결을 위해서 총력을 경주해야 하는데 오늘 보고된 내용은 너무 현대측에 실질적으로 모든 것을 일임하고 있지 않느냐 하는 인상을 지울 수 없는데 좀더 총체적이고 강력한 석방대책을 세워서 추진해야 된다고 보는데 장관께서는 어떻게 생각하십니까? 간략하게 답변 주시지요. [SEP]
['[CLS]', '금', '##강', '##산', '관

In [29]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   8928,  47181,  21386,   8900, 118649, 118617,   9311,
        30858,  22458,   9516,   9547,  46520,  12945,  71439,  10892,
         9604,  46766,  14153,   9414, 103088,   9770,  53635,   9689,
        11664,   9647, 119081,  48345,    119,    100,    122,  10954,
          125,  21789,  38851,  19105,  10530,   9144,  90387,   9485,
       101440,  10530,   9365, 118824, 119436,  85634,    100,    119,
         9604,  44130,   9144,   9638,   9297,  17730,  14523,  74322,
        10622,  19905,  12424,   9764,  27355,  18471,  17022,   9761,
        33975,   8885,  16323, 108436,  16139,  11664,    100,    119,
        21890,   9604,  44130,   8893, 118642,  18108,   9130,  68833,
         9356,  14867,    100,  20729,  80956,   9547,  46520,  12945,
        71439,  10739,   9949,  25486,  17022,   9641,  54305,  30050,
        45021,  11664,    100,    119,   9596, 118728,   9952,  14867,
         9604,  44130,   9764,  50248,   9460,  10954,  41886,   9366,
      

In [30]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [31]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,   8928,  47181,  21386,   8900, 118649, 118617,   9311,  30858,
         22458,   9516,   9547,  46520,  12945,  71439,  10892,   9604,  46766,
         14153,   9414, 103088,   9770,  53635,   9689,  11664,   9647, 119081,
         48345,    119,    100,    122,  10954,    125,  21789,  38851,  19105,
         10530,   9144,  90387,   9485, 101440,  10530,   9365, 118824, 119436,
         85634,    100,    119,   9604,  44130,   9144,   9638,   9297,  17730,
         14523,  74322,  10622,  19905,  12424,   9764,  27355,  18471,  17022,
          9761,  33975,   8885,  16323, 108436,  16139,  11664,    100,    119,
         21890,   9604,  44130,   8893, 118642,  18108,   9130,  68833,   9356,
         14867,    100,  20729,  80956,   9547,  46520,  12945,  71439,  10739,
          9949,  25486,  17022,   9641,  54305,  30050,  45021,  11664,    100,
           119,   9596, 118728,   9952,  14867,   9604,  44130,   9764,  50248,
          9460,  10954,  41886,   9366, 

In [32]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

모델 생성


In [33]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [34]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


모델 학습

In [35]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):

    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [36]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))

    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [38]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(DEVICE) for t in batch)

        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(DEVICE) for t in batch)

        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # 그래디언트 계산 안함
        with torch.no_grad():
            # Forward 수행
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        # 출력 로짓 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   500  of  9,048.    Elapsed: 0:05:42.
  Batch 1,000  of  9,048.    Elapsed: 0:11:25.
  Batch 1,500  of  9,048.    Elapsed: 0:17:08.
  Batch 2,000  of  9,048.    Elapsed: 0:22:51.
  Batch 2,500  of  9,048.    Elapsed: 0:28:34.


KeyboardInterrupt: ignored

평가

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)

    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch

    # 그래디언트 계산 안함
    with torch.no_grad():
        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    # 출력 로짓 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

새로운 문장 평가

In [ ]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [ ]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)

    # 그래디언트 계산 안함
    with torch.no_grad():
        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    # 출력 로짓 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [ ]:
logits = test_sentences(['---문장---'])

print(logits)
print(np.argmax(logits))